# Objective

- Building a relatively-lightweight CNN to classify images into either interior or exterior images

In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from PIL import Image


DATA_DIR = "/kaggle/input/interior-exterior-scene-classification"
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

# Load image paths and labels
image_paths = []
labels = []

label_map = {"Interior": 0, "Exterior": 1}

for label_name in ["Interior", "Exterior"]:
    folder_path = os.path.join(DATA_DIR, label_name)
    for fname in os.listdir(folder_path):
        if fname.endswith(".png"):
            image_paths.append(os.path.join(folder_path, fname))
            labels.append(label_map[label_name])

# Train-test split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Preprocessing function
def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0  # Normalise to between 0 and 1
    return image, label

# Build TensorFlow Datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

train_ds = train_ds.map(load_and_preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(load_and_preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Build base model (MobileNetV2 backbone)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

# Build the top model
top_model = models.Sequential([
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")
])

# Combine the base and the top
model = models.Sequential([
    base_model,
    top_model
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# Evaluate final accuracy
loss, acc = model.evaluate(val_ds)
print(f"\nFinal validation accuracy: {acc:.4f}")


2025-07-04 12:43:42.360626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751633022.563315      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751633022.624151      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 12:43:57.675045: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 36s 918ms/step - accuracy: 0.7401 - loss: 0.5405 - val_accuracy: 0.9545 - val_loss: 0.1646
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 807ms/step - accuracy: 0.9548 - loss: 0.1220 - val_accuracy: 0.9394 - val_loss: 0.1491
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 830ms/step - accuracy: 0.9801 - loss: 0.0653 - val_accuracy: 0.9444 - val_loss: 0.1458
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 823ms/step - accuracy: 0.9944 - loss: 0.0428 - val_accuracy: 0.9545 - val_loss: 0.1401
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 804ms/step - accuracy: 0.9911 - loss: 0.0290 - val_accuracy: 0.9545 - val_loss: 0.1517
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 816ms/step - accuracy: 0.9907 - loss: 0.0255 - val_accuracy: 0.9646 - val_loss: 0.1497
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 825ms/step - accuracy: 0.9972 - loss: 0.0178 - val_accuracy: 0.9596 - val_loss: 0.1432
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 793ms/

In [3]:
top_model.save("top_classifier_head.h5")